In [ ]:
import random
import pickle
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [ ]:
text_df = pd.read_csv('fake_or_real_news.csv')

In [ ]:
text = list(text_df.text.values)
joined_text = " ".join(text)

In [ ]:
partial_text = joined_text[:200000]

In [ ]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [ ]:
unique_tokens = np.unique(tokens)
unique_tokens_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [ ]:
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens) - n_words):
  input_words.append(tokens[i:i + n_words])
  next_words.append(tokens[i + n_words])

In [ ]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [ ]:
for i, words in enumerate(input_words):
  for j, word in enumerate(words):
    X[i, j, unique_tokens_index[word]] = 1
  y[i, unique_tokens_index[next_words[i]]] = 1

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01), metrics=['accuracy'])
model.fit(X, y, batch_size=128, epochs=30, shuffle=True)

Epoch 1/30
267/267 [==============================] - 9s 18ms/step - loss: 7.0560 - accuracy: 0.0564
Epoch 2/30
267/267 [==============================] - 5s 18ms/step - loss: 6.8293 - accuracy: 0.0650
Epoch 3/30
267/267 [==============================] - 5s 17ms/step - loss: 6.6151 - accuracy: 0.0752
Epoch 4/30
267/267 [==============================] - 5s 17ms/step - loss: 6.3746 - accuracy: 0.0938
Epoch 5/30
267/267 [==============================] - 5s 18ms/step - loss: 6.1416 - accuracy: 0.1100
Epoch 6/30
267/267 [==============================] - 4s 16ms/step - loss: 5.9006 - accuracy: 0.1263
Epoch 7/30
267/267 [==============================] - 4s 17ms/step - loss: 5.6453 - accuracy: 0.1420
Epoch 8/30
267/267 [==============================] - 5s 19ms/step - loss: 5.3607 - accuracy: 0.1631
Epoch 9/30
267/267 [==============================] - 5s 17ms/step - loss: 5.0749 - accuracy: 0.1881
Epoch 10/30
267/267 [==============================] - 4s 16ms/step - loss: 4.7815 - accura

In [ ]:
model.save('predict-next-word-model')

In [ ]:
model = load_model('predict-next-word-model')

In [ ]:
def generate_text(input_text, text_length, creativity=3):
  word_sequence = input_text.split()
  current = 0
  for _ in range(text_length):
    sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
    try:
      choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
    except:
      choice = random.choice(unique_tokens)
    word_sequence.append(choice)
    current += 1
  return " ".join(word_sequence)

In [ ]:
generate_text("Mr president will go to some of the country that", 100, 5)

'Mr president will go to some of the country that launchers slogans weiner lofty catch all bias 1990 japan quickly promptly operating idiot failed factors define mind goods continue demons taliban lousy cutting sunni early rajan posts realclearpolitics troop debate chart enriching board oregon discouraged readiness right snubs bush computers cheer stress called discovered collars rice dec dad newsletter exacerbated absolute pointed voices truly wholly guns tycho pooh projecting flight disposal revolt majority fuelled commonality scared version puzzle night dan vs wings helicopters miss ethnicity convention nine further pt professional grinding flags stepping unnoticed ronald angry marked filed suggesting rest lewandowski comprehend edged dismissed prevalence lousy adopting closely drawing executives'